# Data

In [ ]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Define folder paths
output_folder = "mnist_sample_data"
train_folder = os.path.join(output_folder, "train")
test_folder = os.path.join(output_folder, "test")

# Create directories
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Set sample size
train_sample_size = 500  # Number of training images
test_sample_size = 500   # Number of testing images

# Save a subset of training images
for i, (image, label) in enumerate(zip(x_train[:train_sample_size], y_train[:train_sample_size])):
    label_folder = os.path.join(train_folder, str(label))
    os.makedirs(label_folder, exist_ok=True)
    image_path = os.path.join(label_folder, f"{i}.png")
    Image.fromarray(image).save(image_path)

# Save a subset of test images
for i, (image, label) in enumerate(zip(x_test[:test_sample_size], y_test[:test_sample_size])):
    label_folder = os.path.join(test_folder, str(label))
    os.makedirs(label_folder, exist_ok=True)
    image_path = os.path.join(label_folder, f"{i}.png")
    Image.fromarray(image).save(image_path)

print(f"MNIST sample dataset (1000 images) has been saved in '{output_folder}' folder.")

# Load from a Directory

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Define data paths
train_dir = "mnist_sample_data/train"
test_dir = "mnist_sample_data/test"

# Define hyperparameters
BATCH_SIZE = 32
IMG_SIZE = (128, 128)  # Resize images for transfer learning

# Load training and validation datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

# Data Augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# Apply prefetching for performance optimization
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y)).prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

In [ ]:
# Load MobileNetV2 as a base model (Transfer Learning)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SIZE + (3,), 
                                               include_top=False,  # Remove fully connected layers
                                               weights="imagenet")
base_model.trainable = False  # Freeze the base model

# Build the Transfer Learning Model
# Define Input Layer
inputs = keras.Input(shape=(128, 128, 3))
x = base_model(inputs, training=False)  # Apply Pretrained Model # Ensure batch norm layers don't update
x = layers.GlobalAveragePooling2D()(x) # Global Average Pooling
x = layers.Dense(128, activation="relu")(x) # Fully Connected Layers
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(10, activation="softmax")(x) # Output Layer (10 Classes)

# Create Model
model = keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(train_ds, validation_data=test_ds, epochs=3)

# Plot training results
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.title("Training Accuracy vs Validation Accuracy")
plt.show()


In [ ]:
y_test = np.concatenate([y for _, y in test_ds])

In [ ]:
y_pred = model.predict(test_ds)
y_pred = np.argmax(y_pred, axis = 1)

from sklearn.metrics import f1_score, classification_report
report = classification_report(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')

# Print classification reports
print("Classification Report:\n", report)
print("F1 Macro Score:", f1_macro)
print("--"*10)